play with .bam file with pysam 

In [1]:
import pysam
import os
basePath = "/mnt/archgen/users/yilei/Data/iberian_BAM"
id = "I10280"
path2bam = f"{basePath}/{id}_1240k.bam"
#path2bam = basePath + f"/{id}/{id}_1240k.bam"


In [2]:
sf = pysam.AlignmentFile(path2bam, "rb")

In [15]:
import h5py
import numpy as np
import gzip
import time
# compare mismatch rate at variable and non-variable sites
# grep all variable sites on chrX from 1240k panel
f = h5py.File('/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chrX.hdf5', 'r')
pos = f['variants/POS']
rec = f['variants/MAP']
ref = f['variants/REF'][:]
alt = f['variants/ALT'][:, 0]
assert(len(ref) == len(pos))
assert(len(alt) == len(pos))
# 1 here means there is just one sample
# 2 : [ref read count, alt read count]
ad = np.zeros((len(pos), 1, 2))

#use ANGSD's HapMapChrX instead?
# pos = []
# with gzip.open('/mnt/archgen/users/yilei/bin/angsd/RES/HapMapChrX.gz') as file:
#     for line in file:
#         phy, *_ = line.strip().split()
#         pos.append(int(phy))
# print(f'number of SNP sites to check: {len(pos)}')
pos = np.unique(np.array(pos))
pos_pysam = pos-1 # pysam's use 0-based coordinate system

major_adj = 0
minor_adj = 0
major_foc = 0
minor_foc = 0
base2index = {'A':0, 'C':1, 'G':2, 'T':3}
flip = {'A':'T', 'T':'A', 'G':'C', 'C':'G'}
for i, p in enumerate(pos_pysam):
    #print(f'###### focal site: {p} #######')
    for pileupcolumn in sf.pileup('X', p-4, p+5, min_mapping_quality=30, min_base_quality=20, truncate=True):
        basePos = pileupcolumn.pos
        cov = 0
        rc = np.zeros(4)
        #print(f'coverage at base {pileupcolumn.pos}: {pileupcolumn.n}')
        for pileupread in pileupcolumn.pileups:
            if not pileupread.is_del and not pileupread.is_refskip:
                baseCall = pileupread.alignment.query_sequence[pileupread.query_position]
                #if pileupread.alignment.is_reverse: # do we need this ?!
                #    baseCall = flip[baseCall]
                rc[base2index[baseCall]] += 1
                cov += 1
                # if basePos == p:
                #     if baseCall == ref[i]:
                #         ad[i, 0, 0] += 1
                #     elif baseCall == alt[i]:
                #         ad[i, 0, 1] += 1
                #     else:
                #         if (baseCall == 'T' and 'C' in [ref[i], alt[i]]) or (baseCall == 'A' and 'G' in [ref[i], alt[i]]):
                #             print(f'potential DNA damage pattern')
                #         print(f'at position {pos[i]} the read {baseCall} is neither ref {ref[i]} nor alt {alt[i]}.')

        if cov == 1:
            continue # sites covered by only 1 read is not informative for estimating genotyping error
        if basePos != p:
            major_adj += np.max(rc)
            minor_adj += np.sum(rc) - np.max(rc)
        else:
            major_foc += np.max(rc)
            minor_foc += np.sum(rc) - np.max(rc)
        if np.sum(rc == 0) < 2:
            print(f'more than 2 genotype calls at {basePos}')

print(f'number of major reads at flanking sites: {major_adj}')
print(f'number of minor reads at flanking sites: {minor_adj}')
print(f'number of major reads at focal sites: {major_foc}')
print(f'number of minor reads at focal sites: {minor_foc}')

print(f'err rate at flanking sites: {minor_adj/(minor_adj + major_adj)}')
print(f'err rate at focal sites: {minor_foc/(minor_foc + major_foc)}')


number of major reads at flanking sites: 19899.0
number of minor reads at flanking sites: 28.0
number of major reads at focal sites: 2482.0
number of minor reads at focal sites: 6.0
err rate at flanking sites: 0.0014051287198273699
err rate at focal sites: 0.002411575562700965


In [6]:
# run hapCon on the hdf5 file made from bam
# compare this result to ANGSD estimate, does it make sense?
import os

l = len(pos)
k = 1 # 1 bam = 1 sample
rec = f['variants/MAP']
gt = np.zeros((l, k, 2)) # dummy gt matrix, won't be used for inference
dt = h5py.special_dtype(vlen=str)  # To have no problem with saving

path2save = f'{basePath}/{id}/{id}.hdf5'
if os.path.exists(path2save):  # Do a Deletion of existing File there
    os.remove(path2save)


with h5py.File(f'{basePath}/{id}/{id}.hdf5', 'w') as f0:
    # Create all the Groups
    f_map = f0.create_dataset("variants/MAP", (l,), dtype='f')
    f_ad = f0.create_dataset("calldata/AD", (l, k, 2), dtype='i')
    f_ref = f0.create_dataset("variants/REF", (l,), dtype=dt)
    f_alt = f0.create_dataset("variants/ALT", (l,), dtype=dt)
    f_pos = f0.create_dataset("variants/POS", (l,), dtype='i')
    f_gt = f0.create_dataset("calldata/GT", (l, k, 2), dtype='i')
    f_samples = f0.create_dataset("samples", (k,), dtype=dt)

    # Save the Data
    f_map[:] = rec
    f_ad[:] = ad
    f_ref[:] = ref.astype("S1")
    f_alt[:] = alt.astype("S1")
    f_pos[:] = pos
    f_gt[:] = gt
    f_samples[:] = np.array([id]).astype("S10")

In [1]:
import sys
sys.path.insert(0, "/mnt/archgen/users/yilei/tools/hapROH/package")  # hack to get local package first in path [FROM HARALD - DELETE!!!]
from hapsburg.PackagesSupport.hapsburg_run import hapsb_ind  # Need this import
from hapsburg.PackagesSupport.hapsburg_run import hapCon_chrom  # Need this import
from hapsburg.PackagesSupport.hapsburg_run import hapCon_chrom_2d  # Need this import
import numpy as np

In [10]:
hapCon_chrom(id, ch='X', 
            path_targets=f"{basePath}/{id}/{id}.hdf5",
            h5_path1000g='/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr',
            meta_path_ref='/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv', 
            folder_out="./test_output/contaminate/first_try/", prefix_out="fast_",
            e_model="readcount_contam", p_model="SardHDF5", post_model="Standard", save=True, save_fp=False, 
            n_ref=2504, diploid_ref=True, exclude_pops=[], conPop=[], readcounts=True, random_allele=False,
            c=np.arange(0, 0.2, 0.005), roh_in=1, roh_out=0, roh_jump=300, e_rate=0.002, e_rate_ref=1e-3, 
            cutoff_post = 0.999, max_gap=0, roh_min_l = 0.01, logfile=False)

Using Rescaled HMM.
Loaded Pre Processing Model: SardHDF5
Loading Individual: I10939

Loaded 47898 variants
Loaded 1 individuals
HDF5 loaded from /mnt/archgen/users/yilei/Data/iberian_BAM/I10939/I10939.hdf5

Loaded 47898 variants
Loaded 2504 individuals
HDF5 loaded from /mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chrX.hdf5

Intersection on Positions: 47895
Nr of Matching Refs: 47895 / 47895
Ref/Alt Allele Matching: 47895 / 47895
Flipped Ref/Alt Alleles for 0 SNPs
Together: 47895 / 47895
2504 / 2504 Individuals included in Reference
Extracting up to 2504 Individuals
Exctraction of hdf5 done. Subsetting...!
size of gts matrix: (2, 47895)
Extraction of 2 Haplotypes complete
Flipping Ref/Alt Allele in target for 0 SNPs...
Exctraction of hdf5 done. Subsetting...!
size of gts matrix: (5008, 47895)
Extraction of 3775 Haplotypes complete
Subset to markers with data: 6857 / 47895
Fraction SNPs covered: 0.1432
Successfully saved target individual data to: ./test_output/contaminate

([-1243.4409089360374,
  -1233.4201624139791,
  -1230.461253467486,
  -1230.4580372272158,
  -1232.1560409018175,
  -1234.983206877424,
  -1238.6251637593614,
  -1242.8886343347826,
  -1247.6454285136272,
  -1252.8057197738444,
  -1258.3038772593322,
  -1264.0903443494144,
  -1270.1266931010155,
  -1276.3824635621506,
  -1282.8330622583221,
  -1289.4583179758497,
  -1296.2414610453611,
  -1303.1683843488206,
  -1310.2270969734157,
  -1317.4073128040525,
  -1324.700135652764,
  -1332.0978147595197,
  -1339.5935524575607,
  -1347.181351093628,
  -1354.8558898930407,
  -1362.6124249516154,
  -1370.4467072911843,
  -1378.3549151702568,
  -1386.333597751424,
  -1394.3796278957348,
  -1402.4901623514634,
  -1410.6626079784735,
  -1418.8945929333481,
  -1427.1839419582207,
  -1435.5286550847457,
  -1443.9268891961647,
  -1452.376941993818,
  -1460.8772379964253,
  -1469.4263162658208,
  -1478.0228196053654],
 0.015,
 0.00738069299475812,
 0.022619307005241878)

In [4]:
hapCon_chrom_2d(id, ch='X', 
            path_targets=f"{basePath}/{id}/{id}.hdf5",
            h5_path1000g='/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr',
            meta_path_ref='/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv', 
            folder_out="./test_output/contaminate/first_try/", prefix_out="fast_",
            e_model="readcount_contam", p_model="SardHDF5", post_model="Standard", save=True, save_fp=False, 
            n_ref=2504, diploid_ref=True, exclude_pops=[], conPop=[], readcounts=True, random_allele=False,
            c=0.025, roh_in=1, roh_out=0, roh_jump=300, e_rate=0.01, e_rate_ref=1e-3, 
            cutoff_post = 0.999, max_gap=0, roh_min_l = 0.01, logfile=False)

Using Rescaled HMM.
Loaded Pre Processing Model: SardHDF5
Loading Individual: I10939

Loaded 47898 variants
Loaded 1 individuals
HDF5 loaded from /mnt/archgen/users/yilei/Data/iberian_BAM/I10939/I10939.hdf5

Loaded 47898 variants
Loaded 2504 individuals
HDF5 loaded from /mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chrX.hdf5

Intersection on Positions: 47895
Nr of Matching Refs: 47895 / 47895
Ref/Alt Allele Matching: 47895 / 47895
Flipped Ref/Alt Alleles for 0 SNPs
Together: 47895 / 47895
2504 / 2504 Individuals included in Reference
Extracting up to 2504 Individuals
Exctraction of hdf5 done. Subsetting...!
size of gts matrix: (2, 47895)
Extraction of 2 Haplotypes complete
Flipping Ref/Alt Allele in target for 0 SNPs...
Exctraction of hdf5 done. Subsetting...!
size of gts matrix: (5008, 47895)
Extraction of 3775 Haplotypes complete
Subset to markers with data: 6857 / 47895
Fraction SNPs covered: 0.1432
Successfully saved target individual data to: ./test_output/contaminate

(array([0.01850147, 0.        ]), array([nan, nan]))

In [5]:
f = h5py.File('/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chrX.hdf5', 'r')
pos = f['variants/POS']
rec = f['variants/MAP']
refs = f['variants/REF'][:]
alts = f['variants/ALT'][:, 0]
gts = np.array(f["calldata/GT"])
nloci, nind, _ = gts.shape
gts = gts.reshape(nloci, nind*2)
# exclude haplotypes with missing genotypes (for male x chromosome, one column is always -1, for example)
gts = gts.astype(np.float32)
gts[gts == -1] = np.nan
freqs = np.nanmean(gts, axis=1)
assert(len(pos) == len(refs))
assert(len(pos) == len(alts))
assert(len(pos) == len(freqs))

# write a bed file for the focal site, note that bed file uses 0-based coordinate
pos = np.array(pos)
print(np.where(pos == 47635874))


# pos = pos-1
# with open('/mnt/archgen/users/yilei/bin/angsd/RES/1240kChrX.bed', 'w') as out:
#     for p in pos:
#         out.write(f'X\t{p}\t{1+p}\n')



# with open('/mnt/archgen/users/yilei/bin/angsd/RES/1240kChrX.txt','w') as out:
#     for p, ref, freq, alt in zip(pos, refs, freqs, alts):
#         out.write(f'{p}\t{ref}\t{freq}\t+\t{alt}\n')


(array([17343, 17344]),)


In [ ]:
sys.path.insert(0, "/mnt/archgen/users/yilei/tools/hapROH/package")
from hapsburg.PackagesSupport.hapsburg_run import hapCon_chrom_BFGS
from hapsburg.PackagesSupport.hapsburg_run import hapCon_chrom_2d

mle, low95, up95 = hapCon_chrom_BFGS("I0843", 'X', save=False, save_fp=False, n_ref=2504, diploid_ref=True, 
        exclude_pops=[], conPop=[], e_model="readcount_contam", p_model="SardHDF5", 
        readcounts=True, random_allele=False, post_model="Standard", 
        path_targets = '/mnt/archgen/users/yilei/Data/iberian_BAM/HDF5/I0843_1240k.hdf5', 
        folder_out='/mnt/archgen/users/yilei/Data/iberian_BAM/hapCon/',
        h5_path1000g='/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr',
        meta_path_ref='/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv', 
        prefix_out="", c=0.025, roh_in=1, roh_out=0, roh_jump=300, e_rate=0.0019, e_rate_ref=0.0,
        max_gap=0, cutoff_post = 0.999, roh_min_l = 0.01, logfile=False)
print(f'BFGS estimate: {mle}({low95} - {up95})')